In [1]:
%matplotlib inline
import numpy as np
import sys
import scipy
sys.path.append('../datasets')
sys.path.append('../optimization')
sys.path.append('../util')
from eval import compute_weighted_loss, compute_loss_k

In [2]:
from sentiment_analysis import sentiment_analysis_data, sm_problem
datadir = '/data/sentiment_analysis/domain_adaptation/'

In [3]:
from scipy.optimize import minimize, fmin_slsqp
from compute_quantities import compute_u, compute_v, compute_grad_v

def equality_constraint(x):
    z = x[:-1]
    return sum(z) - 1

def bnd(p):
    b = []
    for k in range(p):
        b.append((0,1))
    b.append((-np.inf, np.inf)) # no bound on gamma
    return b

def u_minus_v(z0, DP):
    u = compute_u(z0, DP)
    v = compute_v(z0, DP)
    return u-v

def main_constraint(x, x0, DP):
    gamma = x[-1]
    z = np.squeeze(np.array(x[:-1]))
    z0 = np.squeeze(np.array(x0[:-1]))
    u_z = compute_u(z, DP)
    v_z0 = compute_v(z0, DP)
    gv_z0 = compute_grad_v(z0, DP)
    return gamma - (u_z - v_z0 - np.squeeze(np.array((z-z0)*gv_z0)))

def main_constraint_per_k(x, x0, DP, k):
    gamma = x[-1]
    z = np.squeeze(np.array(x[:-1]))
    z0 = np.squeeze(np.array(x0[:-1]))
    u_z = compute_u(z, DP)
    v_z0 = compute_v(z0, DP)
    gv_z0 = compute_grad_v(z0, DP)
    return gamma - (u_z[k] - v_z0[k] - np.array((z-z0)*gv_z0)[k])


def solve_iter(xp, DP):
    #xx = np.hstack([zp, 0]) #g0 = 0
    fun = lambda x: x[-1] # bottom variable is gamma
    #nonneg_cst = dict(type='ineq', fun=non_negative_constraint)
    eq_cst = dict(type='eq', fun=equality_constraint)
    main_cst = dict(type='ineq', fun=main_constraint, args=(xp, DP))
    cons = [eq_cst]
    for k in range(DP.p):
        cons.append(dict(type='ineq', fun=main_constraint_per_k, args=(xp, DP, k)))
    cons = (eq_cst, main_cst)

    opt = dict(maxiter=1e8, disp=True, eps=1e-5, ftol=1e-10)

    res = minimize(fun, xp, method='SLSQP', constraints=cons, 
                   bounds=bnd(DP.p), options=opt, tol=1e-9)
    print res.message
    return res.x[:-1], res.x[-1], res


def solve_iter2(xp, DP):
#    x0 = np.hstack([zp,0])
    fun = lambda x: x[-1] # bottom variable is gamma
    
    res = fmin_slsqp(fun, xp, eqcons=equality_constraint, 
                     f_ieqcons=main_constraint, 
                     bounds=bnd(DP.p), iter=1e8, disp=1, args=(xp, DP))
    print res.message
    return res.x[:-1], res.x[-1], res

In [4]:
from scipy.optimize import minimize
from compute_quantities import compute_u, compute_v, compute_grad_v
# solve an iteration

#setup constraints: function of [z,gamma]
DP = sm_problem(datadir=datadir, minO=2, ngram=1, 
                split=1, sources=('kitchen', 'dvd', 'books', 'electronics'))
z0 = np.repeat(1.0 / DP.p, DP.p)
uMv = u_minus_v(z0, DP)
g0 = .9 * max(uMv)
x0 = np.hstack([z0, g0])

max_iter = 5
xp = x0
tol = 1e-10
for i in range(max_iter):
    z,gamma,res = solve_iter(xp, DP)
    print 'iter {:d}: z='.format(i), z
    if np.abs(xp[-1] - gamma) < tol:
        print 'absolute change in gamma', xp[-1] - gamma
        break
    xp = np.hstack([z, gamma])

KeyboardInterrupt: 

In [9]:
print gamma
for k in range(DP.p):
    print 'loss h{:d}:'.format(k), compute_loss_k(k,DP)
print 'loss hz:', compute_weighted_loss(z,DP)
print 'loss hz0:', compute_weighted_loss(z0,DP)

-153158463816.0
loss h0: 1.6981646538
loss h1: 1.73481475892
loss h2: 1.70231450794
loss h3: 1.70878277798
loss hz: 1.32289565446
loss hz0: 1.32289684449


In [10]:
z_rand = np.random.rand(DP.p)
z_rand /= z_rand.sum()
print z, z0, z_rand
DP_te = sm_problem(datadir=datadir, minO=2, ngram=1, 
                split=1, sources=('kitchen', 'dvd', 'books', 'electronics'),
                  dset='test')
for k in range(DP_te.p):
    print 'loss h{:d}:'.format(k), compute_loss_k(k,DP_te)
print 'loss hz:', compute_weighted_loss(z,DP_te)
print 'loss hz0:', compute_weighted_loss(z0,DP_te)
print 'loss h_rand', compute_weighted_loss(z_rand, DP_te)

[  3.05910117e-01   3.97589080e-01   4.08099155e-11   2.96500803e-01] [ 0.25  0.25  0.25  0.25] [ 0.33855899  0.21046516  0.02086271  0.43011314]
loss h0: 1.94799571801
loss h1: 2.06471670303
loss h2: 2.04823838554
loss h3: 1.97375490063
loss hz: 1.68345275534
loss hz0: 1.68357482091
loss h_rand 1.68346421243


In [11]:
print 'Only first domain'
ind = range(0,400)
for k in range(DP_te.p):
    print 'loss h{:d}:'.format(k), compute_loss_k(k,DP_te, ind=ind)
print 'loss hz:', compute_weighted_loss(z,DP_te,ind=ind)
print 'loss hz0:', compute_weighted_loss(z0,DP_te, ind=ind)

Only first domain
loss h0: 1.53575310417
loss h1: 2.04999589721
loss h2: 2.19617439067
loss h3: 1.74123563771
loss hz: 1.54131653844
loss hz0: 1.54131653844


In [12]:
print 'Only second domain'
ind = range(400,800)
for k in range(DP_te.p):
    print 'loss h{:d}:'.format(k), compute_loss_k(k,DP_te, ind=ind)
print 'loss hz:', compute_weighted_loss(z,DP_te,ind=ind)
print 'loss hz0:', compute_weighted_loss(z0,DP_te, ind=ind)

Only second domain
loss h0: 2.26082933871
loss h1: 1.84708271182
loss h2: 2.11344494225
loss h3: 2.29882720035
loss hz: 1.8264106291
loss hz0: 1.82641039951


In [13]:
print 'Only third domain'
ind = range(800,1200)
for k in range(DP_te.p):
    print 'loss h{:d}:'.format(k), compute_loss_k(k,DP_te, ind=ind)
print 'loss hz:', compute_weighted_loss(z,DP_te,ind=ind)
print 'loss hz0:', compute_weighted_loss(z0,DP_te, ind=ind)

Only third domain
loss h0: 2.37615242972
loss h1: 2.14577429344
loss h2: 1.66821615181
loss h3: 2.31273246216
loss hz: 1.80578447895
loss hz0: 1.80578447908


In [14]:
print 'Only fourth domain'
ind = range(1200,1600)
for k in range(DP_te.p):
    print 'loss h{:d}:'.format(k), compute_loss_k(k,DP_te, ind=ind)
print 'loss hz:', compute_weighted_loss(z,DP_te,ind=ind)
print 'loss hz0:', compute_weighted_loss(z0,DP_te, ind=ind)

Only fourth domain
loss h0: 1.61924799942
loss h1: 2.21601390964
loss h2: 2.21511805745
loss h3: 1.5422243023
loss hz: 1.56029937487
loss hz0: 1.56078786663


In [13]:
main_constraint(xp, x0, DP)

array([ -3.84172513e+13,  -2.83707886e+13,  -2.83707887e+13,
        -2.87975448e+13])

In [5]:
h = DP.get_regressor()
y = DP.get_true_values()
print h[:10,:]
print y[:10]
print (h[:10,0] - y[:10])**2
print (h[:10,1] - y[:10])**2

[[ 1.58212   1.90002   1.17901   2.11121 ]
 [ 1.95944   2.44088   2.52948   2.60322 ]
 [ 3.89996   4.23462   4.05027   3.84343 ]
 [ 3.89967   2.88608   2.79319   3.23924 ]
 [ 0.899612  0.667825  1.08855   1.30963 ]
 [ 1.09973   2.42364   2.66191   2.47316 ]
 [ 1.89995   0.504526  2.54948   2.56935 ]
 [ 3.63227   2.37233   2.79476   2.91536 ]
 [ 3.89998   3.2775    3.58363   4.42958 ]
 [ 2.03095   2.38629   2.28641   0.811107]]
[ 1.  1.  4.  4.  1.  1.  2.  5.  4.  2.]
[  3.38863694e-01   9.20525114e-01   1.00080016e-02   1.00661089e-02
   1.00777505e-02   9.94607290e-03   1.00100025e-02   1.87068535e+00
   1.00040004e-02   9.57902500e-04]
[ 0.810036    2.07613517  0.05504654  1.24081777  0.11034023  2.02675085
  2.23644248  6.90464963  0.52200625  0.14921996]


In [6]:
import compute_quantities as CQ

In [7]:
z = .25 * np.ones(4)
i = 5079
D = DP.get_marginal_density()
y = DP.get_true_values()
Di = D[i,:]
Dz = CQ.compute_Dz(i, z, DP)
Jz = CQ.compute_Jz(i, z, DP)
Kz = CQ.compute_Kz(i, z, DP)
gz = CQ.compute_gz(i, z, DP)
fz = CQ.compute_fz(i, z, DP)
Fz = CQ.compute_Fz(i, z, DP)
Gz = CQ.compute_Gz(i, z, DP)
hz = Jz / Kz
print Dz, Jz, Kz, hz, fz, gz, Fz, Gz, Di

1.20216792196e-16 4.06236428775e-13 1.56370216792e-13 2.59791434142 3.1975398529e+12 3.1975398529e+12 4.08970444436e+25 4.08970444436e+25 [  3.65122047e-16   3.32392589e-17   1.02270040e-17   7.22788586e-17]


In [38]:
(Jz*1e8 + 1)**2 / (Kz*1e8), (Jz+1)/Kz

(63955.992992263062, 6395079705806.9443)

In [39]:
Jz**2 / Kz

1.0553674443227999e-12

In [40]:
u_minus_v(x0, DP)

array([ 0.,  0.,  0.,  0.])

In [41]:
print 'u', CQ.compute_u(x0[:-1], DP)
print 'v', CQ.compute_v(x0[:-1], DP)

u [  4.09479360e+16   4.09416520e+16   4.09416505e+16   4.09466384e+16]
v [  4.09479360e+16   4.09416520e+16   4.09416505e+16   4.09466384e+16]


In [51]:
D.shape

(6400, 4)

In [49]:
print y[i], Fz
D[i,1] * (Fz)# + 2*gz*y[i]) # + y[i]**2)

2.0 4.08970444436e+25


1359387447.9372702

In [27]:
DP.get_H()

array([ 1.69309  ,  2.383255 ,  4.00707  , ...,  2.7125275,  2.08591  ,
        1.8931275])